# Занятие 10
# Прикладная алгебра и численные методы
## Число обусловленности. Погрешность решения СЛАУ. Погрешность обратной матрицы.

In [1]:
import numpy as np
import sympy
import scipy.linalg
import numpy.linalg
import pandas as pd
from google.colab import files
from IPython.display import Latex

## Число обусловленности
$$
\chi = ||A||\cdot ||A^{-1}||
$$
Число обусловленности вычисляется как произведение норм матрицы $A$ и обратной матрицы $A^{-1}$, число обусловленности зависит от того, какая норма используется.

В numpy.linalg есть функция cond для вычисления числа обусловленности

https://numpy.org/doc/stable/reference/generated/numpy.linalg.cond.html

**linalg.cond(x, p=None)**

Функция cond вычисляет число обусловленности матрицы x, параметр p отвечает за норму, это эквивалент ord в linalg.norm, значения те же самые.
$$
\begin{matrix}
p &	norm\ for\ matrices\\
None &		Frobenius\ norm\\
‘fro’	 &	Frobenius\ norm\\
‘nuc’ &		nuclear\ norm\\
inf &		max(sum(abs(x),\ axis=1))\\
-inf &		min(sum(abs(x),\ axis=1))\\
0	 &	–\\
1	 &	max(sum(abs(x),\ axis=0))\\
-1 &		min(sum(abs(x),\ axis=0))\\
2	 &	2-norm\ (largest\ singular\ value)\\
-2 &		smallest\ singular\ value\\
other &		–
\end{matrix}
$$
## Пример 1.
Узнаем число обусловленности матрицы $M$ в соответствии с матричными нормами $||\cdot||_1$, $||\cdot||_2$, $||\cdot||_\infty$ и $||\cdot||_F$.
$$
\left(
\begin{matrix}
1 & 3 & 5\\
2 & 5 & 9\\
3 & 7 & 11
\end{matrix}
  \right)
$$

In [ ]:
M = [[1, 3, 5], [2, 5, 9], [3, 7, 11]]
print(f'M = {M}', *[f'cond(M, {item})  = {round(np.linalg.cond(M, p=item), 2)}'\
                    for item in (1, 2, np.inf, 'fro')], sep='\n')

M = [[1, 3, 5], [2, 5, 9], [3, 7, 11]]
cond(M, 1)  = 175.0
cond(M, 2)  = 93.48
cond(M, inf)  = 126.0
cond(M, fro)  = 98.59


## Число обусловленности в Sympy
https://docs.sympy.org/latest/modules/matrices/matrices.html#sympy.matrices.matrices.MatrixBase.condition_number

condition_number()

В sympy реализовано минимально возможное число обусловленности, т.е. отношение максимального собственного значения к минимальному, это метод класса матриц Matrix.
## Пример 2
Узнаем число обусловленности матриц $D_2$ и $M_2$ с помощью sympy, сравним с числами обусловленности из numpy.
$$
D_2 = 
\left(
\begin{matrix}
1 & 9\\
-2 & 10
\end{matrix}
  \right),\quad
M_2 = \left(
\begin{matrix}
1 & 9 & 5\\
-2 & 10 & 7\\
-1 & 5 & 4
\end{matrix}
  \right)
$$

In [ ]:
D2 = [[1, 9], [-2, 10]]
M2 = np.asfarray([[1, 9, 5], [-2, 10, 7], [-1, 5, 4]])
for name, matr in zip(('D2', 'M2'), (D2, M2)):
  print(f'{name} = {matr}', *[f'cond(M, {item})  =\
   {round(np.linalg.cond(matr, p=item), 2)}' for item in (1, 2, np.inf, 'fro')],
   sep='\n')
  display(Latex(f'cond\_sympy = \
  {sympy.latex(sympy.Matrix(matr).condition_number().simplify())}'))

D2 = [[1, 9], [-2, 10]]
cond(M, 1)  =   8.14
cond(M, 2)  =   6.49
cond(M, inf)  =   8.14
cond(M, fro)  =   6.64


<IPython.core.display.Latex object>

M2 = [[ 1.  9.  5.]
 [-2. 10.  7.]
 [-1.  5.  4.]]
cond(M, 1)  =   99.43
cond(M, 2)  =   49.64
cond(M, inf)  =   57.0
cond(M, fro)  =   50.67


<IPython.core.display.Latex object>

Не стоит использовать для вычисления числа обусловленности произвольной матрицы  condition_number из sympy, если нет уверенности, что все собственные числа матрицы и обратной матрицы - вещественные числа, лучше - целые.

Матрица M2 была создана на основе farray (np.asfarray([])) для того, чтобы вычисления в sympy проводились приближенные, иначе не удается получить число обусловленности.

Для матрицы $D_2$ удалось получить аналитическое выражение для числа обусловленности (np.asfarray([]) не использовали!).

## Оценка погрешности СЛАУ
## Пример 3
Рассмотрим пример вырожденной СЛАУ $AX = b$ с матрицей $A$ и вектором правой части $b$ такого вида:
$$
A = 
\left(
\begin{matrix}
1 & 2 & 3 & ... & 10\\
1 & 2 & 3 & ... & 10\\
&&&...&\\
1 & 2 & 3 & ... & 10\\
\end{matrix}
  \right), \quad
b = 
\left(
\begin{matrix}
5\\
5\\
...\\
5\\
\end{matrix}
  \right)
$$
Внесем искажения в каждый элемент матрицы $A$ и вектора $b$ в виде случайных чисел типа float (от 0 до 1), умноженных на 0.001.

С помощью np.linalg.solve найдем решение искаженной СЛАУ. 

Очевидно, внося случайные искажения в те же самые исходную вырожденную матрицу $A$ и вектор $b$, в другой раз мы получим другое решение.
Посмотрим, насколько близки решения искаженных СЛАУ, для этого сгенерируем 10 искаженных СЛАУ и сравним их решения. Близость решений будем оценивать с помощью евклидовой нормы разности решений $||X_n - X_m||$, также используем np.allclose. Для каждого случая вычислим также норму разности матриц $||A_n - A_m||$ и векторов правых частей $||b_n - b_m||$.


In [ ]:
rng = np.random.default_rng(12345) # генератор псевдослучайных чисел типа float (от 0 до 1)

X = []
A = []
b = []
for k in range(10):
  Ak = np.array([[i + 1 + 0.001*rng.random() for i in range(10)] for j in range(10)])
  bk = np.array([[5 + 0.001*rng.random()] for i in range(10)])
  X.append(np.round(np.linalg.solve(Ak, bk)[:, 0], 2))
  A.append(Ak)
  b.append(bk)
display(Latex(f'\\text{{решения N 1, 2, 3: }}:\\\\\
{sympy.latex(sympy.Matrix(X[:3]))}'))

for n in range(3):
  for m in range(n + 1, 3):
    print(f'решения {n + 1} и {m + 1} близки? {np.allclose(X[n], X[m])}')
    for name, item in zip(('||Xn - Xm||', '||An - Am||', '||bn - bm||'),
                          (X, A, b)):
        print(f'{name} = {round(np.linalg.norm(item[n] - item[m], 2), 4)}')
        
num_almost_eq = sum([1  for i in range(10) for j in range(10) \
                     if i < j and np.allclose(X[i], X[j], atol=0.8, rtol=0.5)])    
print(f'почти одинаковых решений среди пар из 10 решений: {num_almost_eq}') 

<IPython.core.display.Latex object>

решения 1 и 2 близки? False
||Xn - Xm|| = 3.9262
||An - Am|| = 0.0026
||bn - bm|| = 0.0009
решения 1 и 3 близки? False
||Xn - Xm|| = 4.1041
||An - Am|| = 0.0021
||bn - bm|| = 0.001
решения 2 и 3 близки? False
||Xn - Xm|| = 4.1961
||An - Am|| = 0.0024
||bn - bm|| = 0.0008
почти одинаковых решений среди пар из 10 решений: 1


Получились разные решения. 
# Применение метода главных компонент PCA
## Пример 2
В файле записаны оценки студентов за лабораторные работы и коментарии к ним. Выделим 2 главные компоненты на основе оценок только за лабораторные работы и разобъем студентов на 4 группы в соответствии со знаком каждоый компоненты (++, +-, -+, --), запишем в один файл на отдельные листы с именами "++", "+-", "-+", "--" оценки по всем лабораторным работам, а также округленный накоп и итог. 


In [ ]:
uploaded1 = files.upload()
for fn in uploaded1.keys():
  print(f'User uploaded file "{fn}"')

Saving Ведомость2021_22.xlsx to Ведомость2021_22.xlsx
User uploaded file "Ведомость2021_22.xlsx"


In [ ]:
from sklearn.decomposition import PCA

In [ ]:
n_tests = 18
col_names = [f'ЛР {i + 1} оценка' for i in range(n_tests)] + ['Накоп',
                                                       'итог'] 
df2 = pd.read_excel(fn, sheet_name='алгебра', usecols=col_names)
# заполняем пропуски нулями
marks = set(range(11))
A2 = df2.to_numpy()
n, m = A2.shape
for i in range(n):
    for j in range(m - 2):
        if A2[i, j] not in marks:
            A2[i, j] = 0
# Выделяем 2 главных компоненты
pca_2 = PCA(n_components=2)
pca_2_fit = pca_2.fit_transform(A2[:, :-2])
pca_2_principal_components=pca_2.components_
print(f"""pca_k.explained_variance_ratio_={pca_2.explained_variance_ratio_}
pca_k.mean_={pca_2.mean_}
pca_k.singular_values_={pca_2.singular_values_}""")
# Записываем в отдельные вложенные списки оценки студентов каждой из 4 групп
res_plus_plus = []
res_plus_minus = []
res_minus_plus = []
res_minus_minus = []
for i in range(n):
    first, second = pca_2_fit[i, :]
    if first >= 0 and second >= 0:
        res_plus_plus.append(A2[i, :])
    elif first >= 0 and second < 0:
        res_plus_minus.append(A2[i, :])
    elif first < 0 and second >= 0:
        res_minus_plus.append(A2[i, :])
    else:
        res_minus_minus.append(A2[i, :])    

pca_k.explained_variance_ratio_=[0.50512539 0.07435876]
pca_k.mean_=[5.77777778 5.30409357 4.85964912 5.19883041 4.12865497 1.83625731
 3.30409357 4.61988304 4.66666667 4.4619883  5.30994152 4.54385965
 4.61988304 4.93567251 4.57309942 2.50292398 4.71345029 5.05263158]
pca_k.singular_values_=[144.76205057  55.54196037]


In [ ]:
fname = 'PCA_2.xlsx'
with pd.ExcelWriter(fname) as writer:
    for name, data in zip(("++", "+-", "-+", "--"),
                          (res_plus_plus, res_plus_minus,
                           res_minus_plus, res_minus_minus)):
        df = pd.DataFrame(data, columns=col_names)
        df.to_excel(writer, sheet_name=name, index=True)
        
files.download(fname)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>